In [1]:
import numpy as np
import theano
import theano.tensor as T
import time

import lasagne as L

import sys
sys.path.insert(0, '../rnn_ex/')

from HRED import HRED
from mt_load import load_mt, get_mt_voc, get_w2v_embs
from load_subtle import load_subtle

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
# remember, now the pad value is the same as the <utt_end> token

pad_value = -1 # <utt_end>'s vector is the last one

In [3]:
t0 = time.time()
subtle_path = "/pio/data/data/mtriples/"

train_subtle = load_subtle(subtle_path, split=True, trim=200)
print time.time() - t0

IOError: Failed to interpret file '/pio/data/data/mtriples/Subtle_Dataset.triples.pkl' as a pickle

In [4]:
mt_path = "/pio/data/data/mtriples/"
# mt_path = "/home/maciek/Desktop/mgr/DATA/MovieTriples_Dataset/"

train, valid, test = load_mt(path=mt_path, split=True, trim=200)
idx_to_w, w_to_idx, voc_size, freqs = get_mt_voc(path=mt_path, train_len=len(train))

In [5]:
word2vec_embs, word2vec_embs_mask = get_w2v_embs(path=mt_path)

w2v_train_mask = np.where(word2vec_embs_mask[:,0] == 1)[0]

In [6]:
hred_net = HRED(voc_size=voc_size,
                emb_size=300,
                lv1_rec_size=300, 
                lv2_rec_size=300, 
                out_emb_size=300, 
                num_sampled=200,
                ssoft_probs=freqs,
                emb_init=word2vec_embs)

Building the model...
Compiling theano functions...
Building a network for generating...
Done


In [7]:
hred_net.load_params('trained_models/subtleFixed_300_300_300_300_ssoft200unigr_bs30_cut200_early5.npz')

In [8]:
def print_utt(utt):
    print ' '.join([idx_to_w[x] if x != voc_size-1 else '<utt_end>' for x in utt])

In [9]:
def rnd_next_word(probs, size=1):
    return np.random.choice(np.append(np.arange(probs.shape[0]-1), -1).astype(np.int32), 
                            size=size, p=probs)

def beam_search(beam, dec_init, gamma=-.1, init_seq=np.array([[1]])):
    seq = np.repeat(init_seq.astype(np.int32), beam, axis=0)
#     seq = np.ones((beam, 1), dtype=np.int32)
    probs, dec_init = hred_net.get_probs_and_new_dec_init_fn(seq, dec_init)
    
    words = probs[0].argpartition(-beam)[-beam:].astype(np.int32)
    words[words==voc_size-1] = pad_value
    scores = np.log(probs[0][words])
    seq = np.hstack([seq, words[:, np.newaxis]])
    
    ends = np.zeros(beam)
    
    while not all(ends) and seq.shape[1] < 50:
        probs, dec_init = hred_net.get_probs_and_new_dec_init_fn(seq[:,-1:], dec_init)
        words = probs.argpartition(-beam, axis=1)[:, -beam:].astype(np.int32)
        new_scores = (np.log(probs[np.indices((beam, beam))[0], words]) + scores[:, np.newaxis])
        ###
        new_scores += (new_scores.argsort(axis=1) + 1) * gamma
        ###
        new_scores = new_scores.ravel()
        best = new_scores.argpartition(-beam)[-beam:].astype(np.int32)
        new_seq = []
        new_ends = []
        new_dec_inits = []
        
        for idx in best:
            i,j = divmod(idx, beam)
            new_seq.append(np.concatenate([seq[i], np.array([words[i,j]])]))
            new_dec_inits.append(dec_init[i])
            
            if ends[i]:
                new_ends.append(ends[i])
            elif words[i,j] == w_to_idx['</s>']:
                new_ends.append(seq[i].size)
            else:
                new_ends.append(0)
                
        seq = np.vstack(new_seq)
        ends = np.array(new_ends)
        scores = new_scores[best]
        dec_init = np.vstack(new_dec_inits)
    
    return seq[np.argsort(scores)]

In [26]:
def utt_to_array(utt):
    arr = np.array([w_to_idx[w] for w in utt])[np.newaxis].astype(np.int32)
    arr[arr == -voc_size] = -1
    return arr

def context_summary(context, lookup=True):
    con_init = np.zeros((1, hred_net.lv2_rec_size), dtype=np.float32)
    for utt in context:
        con_init = hred_net.get_new_con_init_fn(utt_to_array(utt) if lookup else utt, con_init)
    return con_init

In [43]:
context = map(lambda x: np.array(x, dtype=np.int32)[np.newaxis], train[9:11])
lookup = False

In [35]:
print_utt(train[11])

<s> why do you look so satisfied ? </s>


In [45]:
context = ['<s> hi . </s>'.split(), '<s> hello , what \' s up ? </s>'.split()]
lookup = True

In [55]:
context = ['<s> yeah , okay . </s>'.split(), '<s> well , i guess i \' ll be going now . </s>'.split()]
lookup = True

In [58]:
beam_size = 10
con_init = context_summary(context, lookup)
W = L.layers.get_all_param_values(hred_net.train_net)[31]
b = L.layers.get_all_param_values(hred_net.train_net)[32]
dec_init = np.repeat(con_init.dot(W) + b, beam_size, axis=0)

for utt in beam_search(beam_size, dec_init, gamma=-10, init_seq=utt_to_array('<s> '.split())):
    print_utt(utt)
    print ''

<s> check turns goes start checks holidays holidays helps reflect helps reflect helps reflect happening stays happening shuts rushes television stays mention strain strain happens asks verge tv holidays checks strain supplies helps explode messes supplies junction checks apologized verge motions asks cheer stops strain checks apologized fails weekends bother

<s> who called checks start bugging supplies strain strain happening stops strain stays happening television stays stays helps weekends strain supplies helps explode messes supplies junction checks strain mention supplies junction sneak weekends strain verge rips rushes shuts cleaners asks sneak verge sacramento messes respond strain stays drift catches weekends

<s> you ought yelled stops coming checks explode respond tv helps bother bother verge stays forth fails helps verge checks apologized fails helps verge checks apologized fails helps verge checks apologized fails weekends bother bother bother bother bother bother investiga

In [10]:
hred_net.train_one_epoch(train_subtle2, 60)

Done 10 batches in 0.86s	training loss:	7.176166
Done 20 batches in 1.69s	training loss:	6.175796
Done 30 batches in 2.44s	training loss:	5.697121
Done 40 batches in 3.47s	training loss:	5.374195
Done 50 batches in 4.27s	training loss:	5.207509
Done 60 batches in 5.11s	training loss:	5.063584
Done 70 batches in 5.92s	training loss:	4.934647
Done 80 batches in 6.69s	training loss:	4.840843


KeyboardInterrupt: 

In [11]:
hred_net.validate(valid, 30)

Done 100 batches in 4.56s
Done 200 batches in 9.25s
Done 300 batches in 14.05s
Done 400 batches in 18.29s
Done 500 batches in 22.79s
Done 600 batches in 27.11s
Done 700 batches in 31.47s
Done 800 batches in 35.77s
Done 900 batches in 40.50s
Done 1000 batches in 45.42s
Done 1100 batches in 50.12s
Done 1200 batches in 54.98s
Done 1300 batches in 59.86s
Done 1400 batches in 64.20s
Done 1500 batches in 68.60s
Done 1600 batches in 73.03s
Done 1700 batches in 77.50s
Done 1800 batches in 82.20s
Done 1900 batches in 87.06s
Done 2000 batches in 91.32s
Done 2100 batches in 96.01s
Done 2200 batches in 100.76s
Done 2300 batches in 105.59s
Done 2400 batches in 110.10s


3.2955217842403752

In [8]:
'''full softmax, bs=30'''
# train, 1 dir, 1 epoch: 3.485554076321884
# val: 3.455356876018342

# train, 2 dir, concat, 1 epoch: 3.4864403798772239
# val: 3.4579001751897063

# train, 2 dir, L2 + concat, 1 epoch: 3.4881669768474675
# val: 3.4584704095551695
# training time: ~4700s

'''sampled softmax'''
# bs=30
# train, 2 dir, L2 + concat, 1 epoch: 3.486180601246621
# val: 3.4811877499289308
# training time: ~2300s

# bs=60
# train, 2 dir, L2 + concat, 1 epoch: 3.5235153449672456
# val: 3.5063306987542759
# training time: ~1900s